In [37]:
import numpy as np
from numpy import *
import matplotlib.pyplot as plt
from scipy.fftpack import fft2, ifft2
from scipy.integrate import solve_ivp

In [38]:
# Part (a)

## Define parameters
tspan = np.arange(0, 4.5, 0.5) 
m = 1
beta = 1
d1 = 0.1
d2 = 0.1
Lx, Ly = 20, 20
nx, ny = 64, 64
N = nx * ny


## Define spatial domain and initial conditions
x2 = np.linspace(-Lx/2, Lx/2, nx + 1)
x = x2[:nx]
y2 = np.linspace(-Ly/2, Ly/2, ny + 1)
y = y2[:ny]
X, Y = np.meshgrid(x, y)

# Computing initial u and v
radius = np.sqrt(X**2 + Y**2)
theta = np.angle(X + 1j * Y)

# Initialize u and v as complex
u0 = np.tanh(radius) * np.cos(m * theta - radius) + 1j * np.zeros((nx, ny))
v0 = np.tanh(radius) * np.sin(m * theta - radius) + 1j * np.zeros((nx, ny))

# Defining A, lambda, omega
def A(x,y):
    return np.sqrt(x**2 + y**2)

def lam(x):
    return 1 - x**2

def w(x):
    return -beta * (x**2)


# Define spectral k values
kx = (2 * np.pi / Lx) * np.concatenate((np.arange(0, nx/2), np.arange(-nx/2, 0)))
kx[0] = 1e-6
ky = (2 * np.pi / Ly) * np.concatenate((np.arange(0, ny/2), np.arange(-ny/2, 0)))
ky[0] = 1e-6
KX, KY = np.meshgrid(kx, ky)
K = KX**2 + KY**2

# RHS of differential equation with u stacked on v
def react_diff_rhs(zt, t, N, nx , ny, K, d1, d2):

    # Recover ut and vt from zt vector and reshape
    ut = zt[:N].reshape((nx, ny))
    vt = zt[N:].reshape((nx, ny))

    # Find u and v in spacial domain
    u = ifft2(ut)
    v = ifft2(vt)

    # Take Laplacian of u and v in spectral domain
    laplace_ut = -K * ut
    laplace_vt = - K * vt

    # Calculate rhs components in spectral domain
    u_rhs_t = fft2(u * lam(A(u,v)) - v * w(A(u,v))) + d1 * laplace_ut
    v_rhs_t = fft2(u * w(A(u,v)) + v * lam(A(u,v))) + d2 * laplace_vt

    # Stack back into vector form
    return np.concatenate([u_rhs_t.reshape(N), v_rhs_t.reshape(N)])

# Solve system of differential equations
zt0 = np.concatenate([fft2(u0).reshape(N), fft2(v0).reshape(N)])
sol = solve_ivp(lambda t, z: react_diff_rhs(z, t, N, nx , ny, K, d1, d2), t_span = [0,4], y0 = zt0, t_eval=tspan)
z_sol = sol.y

A1 = z_sol

print(A1)



    


[[ 24.94003847+0.00000000e+00j  12.73268299-4.59994869e-16j
   -1.38095598-5.43324599e-15j ... -64.02389647-2.71261691e-14j
  -67.76356741-2.10646630e-14j -61.18058974-1.18799894e-14j]
 [-18.55666362-5.81663109e+01j -42.51586944-4.69129224e+01j
  -60.80795253-2.57480390e+01j ... -26.39439597+1.13082890e+02j
    6.86544434+1.23000456e+02j  41.4436393 +1.10055312e+02j]
 [-16.04755868+3.28279829e+01j -22.03971648-4.57977740e+01j
  -23.23089505-1.04141716e+02j ... -25.03391682-9.26527314e+01j
  -29.2936105 -4.09594873e+01j -31.3712619 +1.56986891e+01j]
 ...
 [ 24.73021466-5.66774723e+02j  34.94179045-3.31372917e+02j
   38.82924248-4.97842318e+01j ...   4.99619196+6.02396295e+02j
   -9.93322885+4.90736906e+02j -25.6299042 +2.81792021e+02j]
 [ 25.33720124-3.61633792e+02j  43.00958768-4.53711746e+02j
   51.93221654-4.47841562e+02j ... -30.76392977+2.66442187e+02j
  -58.45411318+4.29165358e+02j -74.0191717 +5.05315322e+02j]
 [ -6.4753501 +3.96245454e+01j  15.86720969-5.83358549e+01j
   37.7389

In [39]:
# Part (b)

def cheb(N):
	if N==0: 
		D = 0.; x = 1.
	else:
		n = arange(0,N+1)
		x = cos(pi*n/N).reshape(N+1,1) 
		c = (hstack(( [2.], ones(N-1), [2.]))*(-1)**n).reshape(N+1,1)
		X = tile(x,(1,N+1))
		dX = X - X.T
		D = dot(c,1./c.T)/(dX+eye(N+1))
		D -= diag(sum(D.T,axis=0))
	return D, x.reshape(N+1)

N = 30
D, x = cheb(N)

D = D/100
x = 10 * x

D[N, :] = 0
D[0, :] = 0
D2 = np.dot(D, D)
y = x

I = np.eye(len(D2))
L = kron(I, D2) + kron(D2, I)

X, Y = np.meshgrid(x, y)

# Computing initial u and v
radius = np.sqrt(X**2 + Y**2)
theta = np.angle(X + 1j * Y)

u0 = np.tanh(radius) * np.cos(m * theta - radius)
v0 = np.tanh(radius) * np.sin(m * theta - radius)
z0 = np.concatenate([u0.flatten(), v0.flatten()])

def react_diff_rhs_cheb(z, t, L, d1, d2):
	u = z[:(N+1)**2]
	v = z[(N+1)**2:]

	u_rhs = lam(A(u,v)) * u - w(A(u,v)) * v + d1 * np.dot(L, u)
	v_rhs = w(A(u,v)) * u + lam(A(u,v)) * v + d2 * np.dot(L, v)

	return(np.concatenate([u_rhs, v_rhs]))


sol = solve_ivp(lambda t, z: react_diff_rhs_cheb(z, t, L, d1, d2), t_span = [0,4], y0 = z0, t_eval=tspan)
z_sol = sol.y

A2 = z_sol
print(A2)

[[ 0.70358468  0.27747296 -0.21642632 ... -0.79934286 -0.41386715
   0.07243133]
 [ 0.73241275  0.32027173 -0.16897659 ... -0.83476263 -0.47193257
   0.00452256]
 [ 0.81058026  0.43052089 -0.05463543 ... -0.88512141 -0.55399867
  -0.08756489]
 ...
 [ 0.58562756  0.90253492  0.99849784 ... -0.46477392 -0.83216814
  -0.9960799 ]
 [ 0.6808609   0.94745156  0.98583757 ... -0.55067639 -0.88167023
  -1.00023994]
 [ 0.71061143  0.96096109  0.97665425 ... -0.60112081 -0.91048363
  -0.99772276]]
